# APMTH 207: Advanced Scientific Computing: 
## Stochastic Methods for Data Analysis, Inference and Optimization
## Homework #5
**Harvard University**<br>
**Spring 2017**<br>
**Instructors: Rahul Dave**<br>
**Due Date: ** Thursday, March 2nd, 2017 at 11:59pm

**Instructions:**

- Upload your final answers as well as your iPython notebook containing all work to Canvas.

- Structure your notebook and your work to maximize readability.

In [49]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats 
from scipy.stats import multivariate_normal
import time
import math
from functools import partial
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

## Problem 1: Monte Carlo and Simulation Revisited
In Homework #2, we used simulation to compute the expected values of functions of random variables. That is, given a random variable $X$, defined over $\mathbb{R}$, distributed according to the pdf $f_X$, and given a real-valued function of $X$, $h(X)$, we approximated $\mathbb{E}[h(X)]$ as follows
$$
\mathbb{E}[h(X)] = \int_{\mathbb{R}} h(x)f_X(x) dx \approx \frac{1}{N} \sum_{i=1}^N h(X_i), \quad X_i \sim f_X
$$

Now, suppose that, instead of being given the distribution $f_X$ and $h(X)$, you were simply asked to evaluate the following complex integral:
$$
I=\int_{0}^{\infty} \frac{x^4\, \sin\left(\sqrt{\ln{(x+1)}}\right)e^{-x}}{2+(x-4)^2} \, dx 
$$
A clever way to apply our Monte Carlo techniques would be to split the integrand as $h(x)f_X(x)$, and then approximate the integral as we have done in Homework #2:
$$
I = \int_{0}^{\infty} h(x)\,f_X(x) dx  \approx \frac{1}{N} \sum\limits_{i=1}^{N} h(X_i)$$ 
where the $X_i$'s are independently drawn from $f_X(x)$. 

We denote the approximation of the integral as follows
$$\hat{I} = \frac{1}{N} \sum\limits_{i=1}^{N} h(X_i), \quad X_i \sim f_X.$$


### Part A:

Rewrite your integrand as a product of two functions, $h(x)g(x)$, which can then be expressed as $h(x)f_X(x)$, where $f_X$ is a pdf (you may use one of the splits we propose in Part B or create your own). Explain why your choice of $h$ is appropriate. Explain why your choice of $g$ is appropriate for creating a pdf $f_X$.

(**Hint:** think about what you would have to do do turn $g$ into a good pdf and $h$ into a function that can be evaluated at multiple samples from this pdf. Think about how to choose these two functions to make your Monte Carlo approximation of $I$ as accurate as possible.)


**Solutions:**

After splitting, your integral will look like
$$
I = \int_{0}^{\infty} h(x) g(x) dx
$$
but you can't do Monte Carlo yet b/c $g$ is not a pdf! So let's turn it into one. First compute the normalizing constant for $g$ (i.e. aread under $g$)
$$
N_g = \int_{\mathbb{R}} g(x) dx
$$
Then define a pdf based on $g$
$$
f_X = \frac{g(x)}{N_g}
$$
So now the integral becomes:
$$
I = N_g \int_{0}^{\infty} h(x) \frac{g(x)}{N_g} dx = N_g \int_{0}^{\infty} h(x) f_X(x) dx = \mathbb{E}_{f_X}[h(X)] \approx \frac{1}{n}\sum_{i=1}^n h(X_i), \quad X_i \sim f_X
$$
**Note:** Don't forget to multiply your tranformed integral by the normalizing constant $N_g$!!!

So generally, you want:
1. $g$ to be easy to integrate, since you need to integrate it to compute $N_g$
2. you want $g$ to be easy to sample from if possible, since you need to sample from it
3. you want $h$ to be flat (small variance) over your $g$ since $Var[\hat{I}] = \frac{Var[h(X)]}{n}$
4. there are a whole bunch of other factors to consider, like where should the probability mass of $g$ be? where $h$ has high variance or low? But we are not requiring this kind of depth.

### Part B:

- Use $\frac{1}{2+(x-4)^2}$ to create your pdf $f_X$. Implement a Metropolis algorithm to sample from $f_X$. Run the simulation 50 times for 150,000 points. Report the value of $\hat{I}$ and that of Var[${\hat{I}}$].


- Use $xe^{-x}$ to create your pdf $f_X$. Implement a Metropolis algorithm to sample from $f_X$. Run the simulation 50 times for 150,000 points. Report the value of $\hat{I}$ and that of Var[${\hat{I}}$].


- Compare the variance of your two estimates. Which choice of $f_X$ is better? Explain why.

**Solution:** 

**BE CAREFUL HERE!!!** You might be tempted to  willy nilly sample $g$ using a normal proposal distribution. But a normal distribution will propose negative values, and the integral is from 0 to infinity, so what do you do with your negative samples??? ***What not to do*** sample a bunch of stuff and then at the end throw away negative values before averaging the values of $h$. This is bad b/c now your truncated chain (the array of samples) is no longer generated by a irreducible markov system (that satisfies detailed balance)!!! The rigorous way to handle this is to re-define your g(x) and h(x) so that they are only supported on non-negative numbers!
$$
\tilde{g}(x) = \begin{cases}
g(x), & x>=0\\
0, & x < 0 
\end{cases}
$$
and 
$$
\tilde{h}(x) = \begin{cases}
h(x), & x>=0\\
0, & x < 0 
\end{cases}
$$
***What not to do*** change the MH algorithm so that it automatically rejects negative proposals. In our special example, hard-coding a rejection in your MH algo is equivalent to defining $g$ to be zero on negative numbers, but hard-coding should be avoided as it can **esaily** correspond to changing the proposal distribution in a way that violates detailed-balance or irreducibility. The point here is that when tweaking M-H, try to tweak the math (the function definitions), NOT the algo (unless you are a sampling ninja and are willing to write proofs).

Again, take Rahul's code with a couple of lines of modification (we're gonna do 10% burn-in and no thinning, starting at x=10, with normal proposal with sigma 10):

In [59]:
p = lambda x: 1. / (2 + (x - 4)**2)
sigma = 10
q = lambda x: np.random.normal(x, sigma, 1)[0]
sims = 50
N = 1.9811

def h_1(x):
    if x >= 0:
        return x**4 * np.sin(np.sqrt(np.log(x + 1))) * np.e**(-x) 
    else:
        return 0
    
def h_2(x):
    if x >= 0:
        return x**3 * np.sin(np.sqrt(np.log(x + 1))) / (2 + (x - 4)**2)
    else:
        return 0
    
def f(x):
    if x >= 0:
        return x**4 * np.sin(np.sqrt(np.log(x + 1))) * np.e**(-x) / (2 + (x - 4)**2)
    else:
        return 0


exp_val, _ = sp.integrate.quad(f, 0, np.inf)

def integrand_h_1(x):
    if x >= 0:
        return (x**4 * np.sin(np.sqrt(np.log(x + 1))) * np.e**(-x) - exp_val)**2/ (2 + (x - 4)**2)
    else:
        return 0
    
def integrand_h_2(x):
    if x >= 0:
        return (x**3 * np.sin(np.sqrt(np.log(x + 1))) / (2 + (x - 4)**2)  - exp_val)**2 * x * np.e**(-x)
    else:
        return 0
    


var, _ = sp.integrate.quad(integrand_h_1, 0, np.inf)

print 'theoretical variance of my monte carlo estimator with first h', var/15000


var, _ = sp.integrate.quad(integrand_h_2, 0, np.inf)

print 'theoretical variance of my monte carlo estimator with second h', var/15000

theoretical variance of my monte carlo estimator with first h 0.00149068527173
theoretical variance of my monte carlo estimator with second h 0.00840078321307


So this is saying that we expect the second $h$ gives us higher variance, which may be surprising, but we'll accept any reasonable interpretation.

In [63]:
N, _ = sp.integrate.quad(lambda x: 1. / (2 + (x - 4)**2), 0, np.inf)
print 'normalization constant for first h', N

normalization constant for first h 1.98114048591


In [66]:
#Rahul's code for Metropolis:

def metropolis(p, qdraw, nsamp, xinit):
    samples=np.empty(nsamp)
    x_prev = xinit
    accepted=0
    for i in range(nsamp):
        x_star = qdraw(x_prev)
        p_star = p(x_star)
        p_prev = p(x_prev)
        pdfratio = p_star/p_prev
        if np.random.uniform() < min(1, pdfratio):
            samples[i] = x_star
            x_prev = x_star
            accepted += 1
        else:#we always get a sample
            samples[i]= x_prev
            
    return samples[15000:], accepted * 1. / len(samples)

def monte_carlo(h, N, p, q):
    x_init = 10
    num_samples = 150000
    samples, accept = metropolis(p, q, num_samples, x_init)
    print 'accept rate:', accept
    return N * np.mean(map(h, list(samples)))
        
    
# THE FIRST SPLIT!!!!!
def p(x):
    if x >= 0:
        return 1. / (2 + (x - 4)**2)
    else:
        return 0
        
sigma = 10
q = lambda x: np.random.normal(x, sigma, 1)[0]
sims = 50

def h(x):
    if x >= 0:
        return x**4 * np.sin(np.sqrt(np.log(x + 1))) * np.e**(-x) 
    else:
        return 0


means = []
for i in range(sims):
    print 'simulation number:', i
    means.append(monte_carlo(h, N, p, q))

    
print np.mean(means)
print np.var(means)


simulation number: 0
accept rate: 0.277453333333
simulation number: 1
accept rate: 0.258313333333
simulation number: 2
accept rate: 0.264933333333
simulation number: 3
accept rate: 0.270706666667
simulation number: 4
accept rate: 0.26402
simulation number: 5
accept rate: 0.268626666667
simulation number: 6
accept rate: 0.263986666667
simulation number: 7
accept rate: 0.2676
simulation number: 8
accept rate: 0.26822
simulation number: 9
accept rate: 0.2676
simulation number: 10
accept rate: 0.268913333333
simulation number: 11
accept rate: 0.267226666667
simulation number: 12
accept rate: 0.266846666667
simulation number: 13
accept rate: 0.27466
simulation number: 14
accept rate: 0.272446666667
simulation number: 15
accept rate: 0.264506666667
simulation number: 16
accept rate: 0.266273333333
simulation number: 17
accept rate: 0.267006666667
simulation number: 18
accept rate: 0.26434
simulation number: 19
accept rate: 0.264093333333
simulation number: 20
accept rate: 0.271926666667
simu

In [67]:
# THE SECOND SPLIT!!!!!
def p(x):
    if x >= 0:
        return x * np.e**(-x)
    else:
        return 0

sigma = 10
q = lambda x: np.random.normal(x, sigma, 1)[0]
sims = 50
N = 1

def h(x):
    if x >= 0:
        return x**3 * np.sin(np.sqrt(np.log(x + 1))) / (2 + (x - 4)**2)
    else:
        return 0


means = []
for i in range(sims):
    print 'simulation number:', i
    means.append(monte_carlo(h, N, p, q))

    
print np.mean(means)
print np.var(means)


simulation number: 0
accept rate: 0.141493333333
simulation number: 1
accept rate: 0.14108
simulation number: 2
accept rate: 0.142986666667
simulation number: 3
accept rate: 0.1417
simulation number: 4
accept rate: 0.140526666667
simulation number: 5
accept rate: 0.143706666667
simulation number: 6
accept rate: 0.142473333333
simulation number: 7
accept rate: 0.142113333333
simulation number: 8
accept rate: 0.145633333333
simulation number: 9
accept rate: 0.143793333333
simulation number: 10
accept rate: 0.143486666667
simulation number: 11
accept rate: 0.143993333333
simulation number: 12
accept rate: 0.143133333333
simulation number: 13
accept rate: 0.141926666667
simulation number: 14
accept rate: 0.144313333333
simulation number: 15
accept rate: 0.14368
simulation number: 16
accept rate: 0.1424
simulation number: 17
accept rate: 0.143533333333
simulation number: 18
accept rate: 0.141213333333
simulation number: 19
accept rate: 0.141573333333
simulation number: 20
accept rate: 0.141

Ok, the variances turned out like we (theoretically expected). Yay!!!

## Problem 2: Metropolis Algorithm

Suppose we ask you to memorize the order of the top five movies on IMDB. When we quiz you on the order afterwards, you might not recall the correct order, but the mistakes you will tend to make in your recall can be modeled by simple probabilistic models.
  
Let's say that the top five movies are:  
1. *The Shawshank Redemption*
2. *The Godfather*
3. *The Godfather: Part II*
4. *The Dark Knight*
5. *Pulp Fiction*

Let's represent this ordering by the vector $\omega = (1,2,3,4,5)$. 

If you were to mistakenly recall the top five movies as:
2. *The Godfather*
3. *The Godfather: Part II*
5. *Pulp Fiction*
4. *The Dark Knight*
1. *The Shawshank Redemption*

We'd represent your answer by the vector $\theta = (2,3,5,4,1)$.

Now, we have a way of quantifying how wrong your answer can be. We define the Hamming distance between two top five rankings, $\theta, \omega$, as follows:
$$d(\theta, \omega) = \sum_{i=1}^5 \mathbb{I}_{\theta_i\neq \omega_i},$$ 
where $\mathbb{I}_{\theta_i\neq \omega_i}$ is the indicator function that returns 1 if $\theta_i\neq \omega_i$, and 0 otherwise.

For example, the Hamming distance between your answer and the correct answer is $d(\theta, \omega)=4$, because you only ranked *The Dark Knight* is correctly. 

Finally, let's suppose that the probability of giving a particular answer (expressed as $\theta$) is modeled as
$$ p(\theta \,|\, \omega, \lambda) \propto  e^{-\lambda\, d(\theta,\, \omega)}.$$

### Part A:

Implement an Metropolis algorithm to produce sample guesses from 500 individuals, with various $\lambda$ values, $\lambda=0.2, 0.5, 1.0$. What are the top five possible guesses?

### Part B:
Compute the probability that *The Shawshank Redemption* is ranked as the top movie (ranked number 1) by the Metropolis algorithm sampler. Compare the resulting probabilities for the various different $\lambda$ values. Summarize your findings.

**Solutions:**

Gonna use the same code for MH, with different targets and proposals and what not.

Note that this proposal,
$$ p(\theta \,|\, \omega, \lambda) \propto  e^{-\lambda\, d(\theta,\, \omega)}.$$
says that the closer (in edit distance) to the correct answer the more likely, more probably, that answer is. So this is a symmetric distribution that peaks at the correct answer.

From the math, you can see right away that $\lambda$ controls the "peakiness" like how concentrated the probability is around the correct answer (like how probable it is that you'd guess a super totally wrong one).


In [94]:
from functools import partial
import collections

def metropolis(p, qdraw, nsamp, xinit, burn):
    samples=[]
    x_prev = xinit
    accepted=0
    for i in range(nsamp):
        x_star = qdraw(x_prev)
        p_star = p(x_star)
        p_prev = p(x_prev)
        pdfratio = p_star/p_prev
        if np.random.uniform() < min(1, pdfratio):
            samples.append(np.array_str(x_star))
            x_prev = x_star
            accepted += 1
        else:#we always get a sample
            samples.append(np.array_str(x_prev))
            
    return samples[int(burn * nsamp):], accepted * 1. / len(samples)


omega = np.array([1, 2, 3, 4, 5])
proposal = lambda theta: np.random.permutation(theta)
d = lambda theta: len((theta - omega)[(theta - omega) != 0])
target_joint = lambda theta, l: np.e**(-l * d(theta))

lambdas = [0.2, 0.5,1.0]

for l in lambdas:
    target = partial(target_joint, l=l)

    x_init = np.array([2, 1, 3, 5, 4])
    print 'lambda = ', l
    num_samples = 500
    samples, accept = metropolis(target, proposal, num_samples, x_init, burn=0)
    print 'accept percent:', accept
    vals, counts = np.unique(samples, return_counts=True)
    top_5 = counts.argsort()[-5:][::-1]
    print 'top 5 guesses:', vals[top_5]
    print 'frequencies:', counts[top_5]
    print '\n\n'

lambda =  0.2
accept percent: 0.874
top 5 guesses: ['[1 2 3 4 5]' '[3 2 1 4 5]' '[5 2 3 4 1]' '[1 2 5 4 3]' '[1 4 2 3 5]']
frequencies: [20 12 11 11 10]



lambda =  0.5
accept percent: 0.658
top 5 guesses: ['[1 2 3 4 5]' '[1 4 3 2 5]' '[1 3 2 4 5]' '[1 2 4 5 3]' '[1 2 5 3 4]']
frequencies: [51 24 17 15 11]



lambda =  1.0
accept percent: 0.44
top 5 guesses: ['[1 2 3 4 5]' '[2 1 3 4 5]' '[1 3 2 4 5]' '[1 2 3 5 4]' '[1 2 5 4 3]']
frequencies: [77 39 31 18 17]





So the larger the lambda, the more likely you're to guess the right answer and the more tightly you're guesses will cluster around the right answer.

To compute $p(\text{shawshank is top} | lambda)$, we count how many guesses have 1 in the first entry.

In [111]:
for l in lambdas:
    target = partial(target_joint, l=l)

    x_init = np.array([2, 1, 3, 5, 4])
    print 'lambda = ', l
    num_samples = 500
    samples, accept = metropolis(target, proposal, num_samples, x_init, burn=0)
    vals, counts = np.unique(samples, return_counts=True)
    top_5 = counts.argsort()[-5:][::-1]
    print 'top 5 guesses:', vals[top_5]
    print 'frequencies:', counts[top_5]
    print 'p(shawshank ranked top | lambda): ', len([s for s in samples if '[1 ' in s])  * 1. / len(samples)
    print '\n\n'
   

lambda =  0.2
top 5 guesses: ['[1 5 3 4 2]' '[1 2 3 4 5]' '[1 4 3 2 5]' '[1 2 3 5 4]' '[3 4 2 1 5]']
frequencies: [15 14 12 10  9]
p(shawshank ranked top | lambda):  0.276



lambda =  0.5
top 5 guesses: ['[1 3 2 4 5]' '[1 2 3 4 5]' '[1 4 3 2 5]' '[1 5 3 4 2]' '[3 2 1 4 5]']
frequencies: [23 17 16 15 14]
p(shawshank ranked top | lambda):  0.294



lambda =  1.0
top 5 guesses: ['[1 2 3 4 5]' '[3 2 1 4 5]' '[1 4 3 2 5]' '[5 2 3 4 1]' '[1 3 2 4 5]']
frequencies: [138  36  26  25  19]
p(shawshank ranked top | lambda):  0.528





So, $p(\text{shawshank is top} | lambda)$ increases as lambda increases.